In [1]:
import pandas as pd
from datetime import date, timedelta

from typing import Mapping, MutableMapping, Sequence, Iterable, List, Set
# from functools import reduce
import funcy

In [2]:
print("hello")
flow_df_spilli1 = pd.read_csv('../spillimacheen/08NA011_QR_Oct-11-2021_12_24_24AM.csv', skiprows=10)
flow_df_spilli2 = pd.read_csv('../spillimacheen/08NA011_QR_Apr-2-2020_06_01_41PM.csv', skiprows = 10)

hello


In [3]:
def format_data(df: pd.DataFrame) -> pd.DataFrame:
    df.index = pd.to_datetime(df['Date (MST)'])
    df = df.resample('60min').mean()
    df = df[df.index.month > 3]
    df = df[df.index.month < 11]
    return df

In [4]:
formatted_spilli1 = format_data(flow_df_spilli1)
formatted_spilli2 = format_data(flow_df_spilli2)


In [5]:
# parsed = [pd.DataFrame(x).describe for x in raw_flows]
# merged = formatted_spilli1.merge(formatted_spilli2, how='left', left_index=True, right_index=True)
merged = pd.concat([formatted_spilli1, formatted_spilli2])

In [6]:
df_snow = pd.read_csv('../spillimacheen/snow.csv')
df_snow.index = pd.to_datetime(df_snow['Timestamp (UTC-06:00)'])

In [7]:
print(merged.shape)
more_merged = merged.merge(df_snow, how='inner', left_index=True, right_index=True)
print(more_merged.shape)

(15411, 2)
(14580, 8)


In [8]:
more_merged.columns


Index(['Parameter ', 'Value (m3/s)', 'Timestamp (UTC-06:00)',
       'Value (Centimetres)', 'Grade Code', 'Approval Level',
       'Interpolation Type', 'Event Timestamp'],
      dtype='object')

In [9]:
trimmed = more_merged.drop(columns=['Parameter ', 'Timestamp (UTC-06:00)',
        'Grade Code', 'Approval Level',
       'Interpolation Type', 'Event Timestamp'])

In [10]:
trimmed.shape

(14580, 2)

In [11]:
trimmed.columns

Index(['Value (m3/s)', 'Value (Centimetres)'], dtype='object')

In [14]:
trimmed.isna().sum()
trimmed.to_csv('./trimmed.csv')

In [13]:
from requests import get

def getWeather(unix_date, lat, long):
    api_key = 'o63RP2YfqXh5YhQynj0l5bsiOH8zPBL48vvZFD7j'
    url = f'https://timemachine.pirateweather.net/forecast/{api_key}/{lat},{long},{unix_date}'
    return get(url)


In [21]:
import numpy as np

In [22]:

datesList = trimmed.resample('24h').mean()

In [25]:
datesList['unix_time'] = datesList.index.astype(np.int64)

/var/folders/x_/wh05yg2n1p70p5wv4hsgwq580000gn/T/ipykernel_16825/1881689952.py:1: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  datesList['unix_time'] = datesList.index.astype(np.int64)


In [26]:
datesList

,Value (m3/s),Value (Centimetres),unix_time
2019-04-01,5.870625,208.800000,1554076800000000000
2019-04-02,6.067326,207.845833,1554163200000000000
2019-04-03,6.427014,207.804167,1554249600000000000
2019-04-04,6.980139,215.175000,1554336000000000000
2019-04-05,7.900069,220.333333,1554422400000000000
...,...,...,...
2021-10-06,21.927083,29.200000,1633478400000000000
2021-10-07,22.530208,43.500000,1633564800000000000
2021-10-08,20.061111,41.708333,1633651200000000000
2021-10-09,18.751042,34.500000,1633737600000000000


In [34]:
max_count = 0
count = 0

lat = 51.17461
lon = -117.21005


time_stamp = datesList['unix_time'].iloc[0]


res = getWeather(time_stamp, lat, lon)


In [35]:
res

<Response [502]>